In [10]:
from google.cloud import storage
import pandas as pd
import pyarrow as pq
import requests


# https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2022-01.parquet

In [18]:
base_url = "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2022-{month:02d}.parquet"
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}

for month in range(1, 13):  # Loop from January (1) to December (12)
    url = base_url.format(month=month)
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        filename = f"green_tripdata_2022-{month:02d}.parquet"
        with open(filename, "wb") as f:
            f.write(response.content)
        print(f"Downloaded {filename}")
    else:
        print(f"Failed to download data for month {month:02d}: HTTP {response.status_code}")



Downloaded green_tripdata_2022-01.parquet
Downloaded green_tripdata_2022-02.parquet
Downloaded green_tripdata_2022-03.parquet
Downloaded green_tripdata_2022-04.parquet
Downloaded green_tripdata_2022-05.parquet
Downloaded green_tripdata_2022-06.parquet
Downloaded green_tripdata_2022-07.parquet
Downloaded green_tripdata_2022-08.parquet
Downloaded green_tripdata_2022-09.parquet
Downloaded green_tripdata_2022-10.parquet
Downloaded green_tripdata_2022-11.parquet
Downloaded green_tripdata_2022-12.parquet


In [8]:
import pyarrow.parquet as pq

files = ["table1.parquet", "table2.parquet"]

schema = pq.ParquetFile(files[0]).schema_arrow
with pq.ParquetWriter("output.parquet", schema=schema) as writer:
    for file in files:
        writer.write_table(pq.read_table(file, schema=schema))

In [ ]:
def upload_blob(bucket_name, source_file_name, destination_blob_name):
    """Uploads a file to the bucket."""
    # The ID of your GCS bucket
    bucket_name = "green_taxi_data_22"
    # The path to your file to upload
    source_file_name = ""
    # The ID of your GCS object
    # destination_blob_name = "storage-object-name"

    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

In [ ]:
    blob.upload_from_filename(source_file_name, if_generation_match=generation_match_precondition)

    print(
        f"File {source_file_name} uploaded to {destination_blob_name}."
    )